In [43]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint
from DataColProc.lesson2.vacancy import get_vacancy_from_hh as dcp_hh
from DataColProc.lesson2.vacancy_sj import get_vacancy_from_sj as dcp_sj

client = MongoClient('127.0.0.1', 27017)

## task 1

In [44]:
def add_vacancies_to_db():
    input_vacancy_name = input('Введите желаемую должность: ').replace(' ', '-')

    db = client[f'vacancies_for_{input_vacancy_name}']
    vacancies = db.vacancies

    vacancy_table_hh = dcp_hh(input_vacancy_name)
    vacancy_table_sj = dcp_sj(input_vacancy_name)

    vacancies_table = pd.concat([vacancy_table_hh, vacancy_table_sj], axis=0, ignore_index=True)
    vacancies.insert_many(vacancies_table.to_dict('records'))

In [45]:
add_vacancies_to_db()


## task 2

In [46]:
def vacancies_find_salary(vacancy_name, salary_value):
    vacancy_name = vacancy_name.replace(' ', '-')
    db = client[f'vacancies_for_{vacancy_name}']
    vacancies = db.vacancies

    return vacancies.find({'$or': [
        {'min_salary': {'$gte': salary_value}},
        {'max_salary': {'$gte': salary_value}}
    ]})

In [47]:
for el in vacancies_find_salary('data scientist', 100000):
    pprint(el)

{'_id': ObjectId('5f623633f71f3feba7fcb5ca'),
 'currency': 'руб',
 'max_salary': 200000.0,
 'min_salary': 180000.0,
 'name': 'Data Scientist (ML/CV)',
 'source': 'https://hh.ru',
 'url': 'https://spb.hh.ru/vacancy/39212305?query=data-scientist'}
{'_id': ObjectId('5f623633f71f3feba7fcb5cc'),
 'currency': 'руб',
 'max_salary': 180000.0,
 'min_salary': 110000.0,
 'name': 'Data Scientist (Remote)',
 'source': 'https://hh.ru',
 'url': 'https://spb.hh.ru/vacancy/39110818?query=data-scientist'}
{'_id': ObjectId('5f623633f71f3feba7fcb5cd'),
 'currency': 'руб',
 'max_salary': 160000.0,
 'min_salary': 120000.0,
 'name': 'Data scientist (middle/junior)',
 'source': 'https://hh.ru',
 'url': 'https://spb.hh.ru/vacancy/38961658?query=data-scientist'}
{'_id': ObjectId('5f623633f71f3feba7fcb5ce'),
 'currency': 'руб',
 'max_salary': 150000.0,
 'min_salary': nan,
 'name': 'Data Scientist / Data analyst',
 'source': 'https://hh.ru',
 'url': 'https://spb.hh.ru/vacancy/39215532?query=data-scientist'}
{'_id

## task 3

In [48]:
def create_or_update_vacancies(vacancy_name):
    vacancy_name = vacancy_name.replace(' ', '-')
    db = client[f'vacancies_for_{vacancy_name}']
    vacancies = db.vacancies

    vacancy_table_hh = dcp_hh(vacancy_name)
    vacancy_table_sj = dcp_sj(vacancy_name)
    vacancies_table = pd.concat([vacancy_table_hh, vacancy_table_sj], axis=0, ignore_index=True)

    list_of_vacancies_in_db_url = []
    for vacancy_from_db in vacancies.find({}, {'_id': 0}):
        list_of_vacancies_in_db_url.append(vacancy_from_db['url'])

    list_of_vacancies_in_table = vacancies_table.to_dict('records')

    for new_vacancy in list_of_vacancies_in_table:
        if new_vacancy['url'] not in list_of_vacancies_in_db_url:
            vacancies.insert_one(new_vacancy)

    print(f'База данных вакансий {"обновлена" if list_of_vacancies_in_db_url else "создана"}!')

In [49]:
create_or_update_vacancies('data scientist')


База данных вакансий обновлена!
